<a href="https://colab.research.google.com/github/IkrashNoman/Understanding-DeepLearning/blob/main/Artificial_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Artificial Neural Networks

In [ ]:
import numpy as np

In [ ]:
def sigmoid(z):
  return 1/(1 + np.exp(-z))

def relu(z):
  return np.maximum(0, z)

def relu_derivative(z):
  return (z>0).astype(int)

In [ ]:
class NeuralNetwork:
  def __init__(self, x_dim, hidden_dim, y_dim):
    np.random.seed(42)
    self.params = {
        "W1": np.random.randn(hidden_dim, x_dim),
        "W2": np.random.randn(y_dim, hidden_dim),
        "b1": np.zeros((hidden_dim, 1)),
        "b2": np.zeros((y_dim, 1)),
    }

  def forward(self, X):
    W1, W2, b1, b2 = self.params["W1"], self.params["W2"], self.params['b1'], self.params['b2']
    z1 = np.dot(W1, X) + b1
    a1 = relu(z1)
    z2 = np.dot(W2, a1) + b2
    y_hat = sigmoid(z2)

    cache = {"Z1": z1, "Z2": z2, "A1": a1, "A2" : y_hat}
    return y_hat, cache

  def backward(self, X, Y, cache):
    m = X.shape[1]
    W1, W2 = self.params["W1"], self.params["W2"]
    A1, A2 =  cache["A1"], cache["A2"]
    z1, z2 = cache["Z1"], cache["Z2"]

    dz2 = A2 - Y
    dw2 = np.dot(dz2, A1.T)/m
    db2 = np.sum(dz2, axis=1, keepdims=True)/m

    da1 = np.dot(W2.T, dz2)
    dz1 = da1 * relu_derivative(z1)
    dw1 = np.dot(dz1, X.T)/m
    db1 = np.sum(dz1, axis=1, keepdims=True)/m

    return {"dw1": dw1, "db1": db1, "dw2": dw2, "db2": db2}

  def update_params(self, grads, lr):
    self.params["W1"] -= lr * grads["dw1"]
    self.params["b1"] -= lr * grads["db1"]
    self.params["W2"] -= lr * grads["dw2"]
    self.params["b2"] -= lr * grads["db2"]

  def train(self, X, Y, epochs= 100000, learning_rate = 0.001):
    for i in range(epochs):
      A2, cache = self.forward(X)
      m = Y.shape[1]
      cost = -np.sum(np.multiply(Y, np.log(A2 + 1e-10)) + np.multiply(1 - Y, np.log(1 - A2 + 1e-10)))/m
      grads = self.backward(X, Y, cache)
      self.update_params(grads, learning_rate)

In [ ]:
X = np.array([
    [0, 0, 0, 0, 1, 1, 1, 1],  # P
    [0, 0, 1, 1, 0, 0, 1, 1],  # Q
    [0, 1, 0, 1, 0, 1, 0, 1]   # R
])
Y = np.array([
    [0, 0, 0, 1, 0, 1, 0, 1]
])

In [ ]:
input_size = X.shape[0]   # 2
hidden_size = 4           # Arbitrary choice
output_size = Y.shape[0]  # 1

# Instantiate and Train
print("Training...")
nn = NeuralNetwork(input_size, hidden_size, output_size)
nn.train(X, Y, epochs=5000, learning_rate=0.1)

# Test
print("\nFinal Predictions:")
predictions, _ = nn.forward(X)
print(predictions)
print("\nRounded Predictions:")
print(np.round(predictions))

Training...

Final Predictions:
[[1.77863132e-04 2.01235405e-02 1.24211446e-04 9.97610689e-01
  7.31908683e-06 9.97833077e-01 1.11235631e-03 9.99984069e-01]]

Rounded Predictions:
[[0. 0. 0. 1. 0. 1. 0. 1.]]
